In [2]:
# Gradio Titanic Hayatta Kalma Tahmini Arayüzü - Çoklu Sayfa
import gradio as gr
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

# Global değişkenler
model = None
scaler = None
final_model_name = None
final_accuracy = None
scaling_required_models = ['Lojistik Regresyon', 'K-En Yakın Komşu', 'Destek Vektör Makinesi']
X_columns = None
training_complete = False

def train_model():
    """Model eğitimi fonksiyonu"""
    global model, scaler, final_model_name, final_accuracy, X_columns, training_complete
    
    try:
        # Veri yükleme ve ön işleme
        df_tit = sns.load_dataset("titanic")
        df_tit_c = df_tit.copy()
        
        # Gereksiz sütunları kaldır
        columns_to_drop = ["embark_town", "who", "adult_male", "deck", "alive", "alone", "class"]
        df_tit_c = df_tit_c.drop(columns_to_drop, axis=1)
        
        # Eksik değerleri doldur
        df_tit_c["age"] = df_tit_c["age"].fillna(round(df_tit_c["age"].mean()))
        df_tit_c["embarked"] = df_tit_c["embarked"].fillna(value="U")
        
        # One-Hot Encoding
        df_tit_c = pd.get_dummies(df_tit_c, columns=["sex", "embarked"], drop_first=True, dtype=int)
        
        # Feature Engineering
        df_tit_c["family_size"] = df_tit_c["sibsp"] + df_tit_c["parch"] + 1
        df_tit_c = df_tit_c.drop(["sibsp", "parch"], axis=1)
        
        # Veri ayrımı
        X = df_tit_c.drop("survived", axis=1)
        y = df_tit_c["survived"]
        X_columns = X.columns
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
        
        # Ölçekleme
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)
        
        # Modeller
        models = {
            'Lojistik Regresyon': LogisticRegression(max_iter=10000, solver="liblinear"),
            'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
            'Karar Ağacı': DecisionTreeClassifier(random_state=42),
            'K-En Yakın Komşu': KNeighborsClassifier(n_neighbors=5),
            'Naive Bayes': GaussianNB(),
            'Destek Vektör Makinesi': SVC(kernel='rbf', random_state=42, probability=True)
        }
        
        # Model değerlendirme ve seçim
        cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        best_score = 0
        results_text = "Model Eğitim Sonuçları:\n" + "="*50 + "\n"
        
        for name, model_obj in models.items():
            if name in scaling_required_models:
                X_train_use = X_train_scaled
                X_test_use = X_test_scaled
            else:
                X_train_use = X_train
                X_test_use = X_test
            
            # Cross validation
            from sklearn.model_selection import cross_val_score
            cv_scores = cross_val_score(model_obj, X_train_use, y_train, cv=cv_strategy, scoring='accuracy')
            
            results_text += f"{name}: CV Ortalama = {cv_scores.mean():.4f}\n"
            
            if cv_scores.mean() > best_score:
                best_score = cv_scores.mean()
                final_model_name = name
                
                # En iyi modeli eğit
                model_obj.fit(X_train_use, y_train)
                y_pred = model_obj.predict(X_test_use)
                final_accuracy = accuracy_score(y_test, y_pred)
                model = model_obj
        
        training_complete = True
        results_text += f"\nEn İyi Model: {final_model_name}\n"
        results_text += f"Test Doğruluğu: {final_accuracy:.4f} ({final_accuracy*100:.2f}%)\n"
        results_text += "\nModel eğitimi başarıyla tamamlandı!"
        
        return results_text
        
    except Exception as e:
        return f"Hata oluştu: {str(e)}"

def predict_survival(pclass, age, fare, sex, sibsp, parch, embarked):
    """Hayatta kalma tahmini fonksiyonu"""
    global model, scaler, final_model_name, final_accuracy, X_columns, training_complete
    
    if not training_complete:
        return "Önce model eğitimi sayfasından modeli eğitmeniz gerekiyor!", None, "Model eğitilmedi"
    
    try:
        # Veri hazırlama
        sex_male = 1 if sex == "Erkek" else 0
        family_size = sibsp + parch + 1
        
        embarked_Q = 1 if embarked == "Queenstown" else 0
        embarked_S = 1 if embarked == "Southampton" else 0  
        embarked_U = 1 if embarked == "Bilinmiyor" else 0
        
        user_df = pd.DataFrame({
            'pclass': [pclass],
            'age': [age],
            'fare': [fare],
            'sex_male': [sex_male],
            'embarked_Q': [embarked_Q],
            'embarked_S': [embarked_S],
            'embarked_U': [embarked_U],
            'family_size': [family_size]
        })
        
        # Tahmin
        if final_model_name in scaling_required_models:
            user_data_scaled = scaler.transform(user_df)
            user_data_scaled_df = pd.DataFrame(user_data_scaled, columns=X_columns)
            prediction_proba = model.predict_proba(user_data_scaled_df)[0]
            prediction = model.predict(user_data_scaled_df)[0]
        else:
            prediction_proba = model.predict_proba(user_df)[0]
            prediction = model.predict(user_df)[0]
        
        survival_chance = prediction_proba[1] * 100
        death_chance = prediction_proba[0] * 100
        
        # Sonuç mesajı
        if prediction == 1:
            result_emoji = "✅"
            result_text = "HAYATTA KALIRSINIZ!"
            if survival_chance > 80:
                detail = "Çok yüksek hayatta kalma şansınız var!"
            elif survival_chance > 60:
                detail = "İyi bir hayatta kalma şansınız var."
            else:
                detail = "Şansınız var ama dikkatli olun."
        else:
            result_emoji = "❌"
            result_text = "HAYATTA KALAMAZSINIZ"
            if death_chance > 80:
                detail = "Çok yüksek risk altındasınız."
            elif death_chance > 60:
                detail = "Yüksek risk altındasınız."
            else:
                detail = "Risk var ama umut da var."
        
        result_message = f"""
        ## 🚢 TİTANİC HAYATTA KALMA TAHMİNİ
        
        **Model:** {final_model_name}  
        **Model Doğruluğu:** {final_accuracy*100:.1f}%
        
        ### 📊 Tahmin Sonuçları:
        - **Hayatta Kalma Şansı:** %{survival_chance:.1f}
        - **Ölüm Riski:** %{death_chance:.1f}
        
        ### {result_emoji} TAHMİN: {result_text}
        {detail}
        
        ### 📋 Girilen Bilgiler:
        - **Sınıf:** {pclass} ({'Birinci' if pclass==1 else 'İkinci' if pclass==2 else 'Üçüncü'})
        - **Yaş:** {age}
        - **Cinsiyet:** {sex}
        - **Kardeş/Eş:** {sibsp}
        - **Ebeveyn/Çocuk:** {parch}
        - **Aile Büyüklüğü:** {family_size} kişi
        - **Bilet Ücreti:** ${fare}
        - **Bindiği Liman:** {embarked}
        """
        
        # Grafik oluşturma
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
        
        categories = ['Hayatta Kalma', 'Ölüm Riski']
        values = [survival_chance, death_chance]
        colors = ['#2E8B57', '#DC143C']
        
        bars = ax.bar(categories, values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.2)
        ax.set_ylabel('Olasılık (%)', fontsize=12)
        ax.set_title(f'Titanic Hayatta Kalma Tahmini\n{result_emoji} {result_text}', fontsize=14, fontweight='bold')
        ax.set_ylim(0, 100)
        
        # Bar üzerine yüzde değerlerini yazma
        for bar, value in zip(bars, values):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 1,
                    f'{value:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')
        
        ax.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        
        return result_message, fig, f"{survival_chance:.1f}%"
        
    except Exception as e:
        return f"Hata oluştu: {str(e)}", None, "Hata"

def create_training_page():
    """Model eğitimi sayfası"""
    with gr.Column():
        gr.Markdown("""
        # 🔧 Model Eğitimi
        ### Titanic veri seti ile makine öğrenmesi modellerini eğitin
        
        Bu sayfada 6 farklı makine öğrenmesi algoritması eğitilir ve en iyi performanslı model otomatik olarak seçilir.
        """)
        
        with gr.Row():
            with gr.Column(scale=1):
                train_button = gr.Button("Modeli Eğit", variant="primary", size="lg")
                
            with gr.Column(scale=2):
                train_output = gr.Textbox(
                    label="Eğitim Sonuçları", 
                    lines=15,
                    interactive=False,
                    placeholder="Eğitim sonuçları burada görünecek..."
                )
        
        gr.Markdown("""
        ### Eğitilen Modeller:
        - **Lojistik Regresyon** 
        - **Random Forest**
        - **Karar Ağacı**
        - **K-En Yakın Komşu**
        - **Naive Bayes**
        - **Destek Vektör Makinesi**
        
        Model eğitimi tamamlandıktan sonra "Tahmin Yap" sekmesine geçebilirsiniz.
        """)
        
        train_button.click(
            fn=train_model,
            outputs=train_output
        )

def create_prediction_page():
    """Tahmin sayfası"""
    with gr.Column():
        gr.Markdown("""
        # 🔮 Hayatta Kalma Tahmini
        ### Kişisel bilgilerinizi girerek Titanic'te hayatta kalma şansınızı öğrenin
        """)
        
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("## 👤 Kişisel Bilgileriniz")
                
                pclass = gr.Dropdown(
                    choices=[1, 2, 3], 
                    label="🎫 Bilet Sınıfı", 
                    value=3,
                    info="1: Birinci Sınıf, 2: İkinci Sınıf, 3: Üçüncü Sınıf"
                )
                
                age = gr.Slider(
                    minimum=0, 
                    maximum=100, 
                    value=25, 
                    label="🎂 Yaş",
                    info="Yaşınızı giriniz"
                )
                
                sex = gr.Radio(
                    choices=["Erkek", "Kadın"], 
                    label="👤 Cinsiyet", 
                    value="Erkek"
                )
                
                fare = gr.Slider(
                    minimum=0, 
                    maximum=500, 
                    value=50, 
                    label="💰 Bilet Ücreti (Dolar)",
                    info="Bilet ücretinizi giriniz"
                )
                
                sibsp = gr.Slider(
                    minimum=0, 
                    maximum=8, 
                    value=0, 
                    label="👫 Kardeş/Eş Sayısı",
                    info="Gemideki kardeş veya eş sayısı"
                )
                
                parch = gr.Slider(
                    minimum=0, 
                    maximum=6, 
                    value=0, 
                    label="👶 Ebeveyn/Çocuk Sayısı",
                    info="Gemideki ebeveyn veya çocuk sayısı"
                )
                
                embarked = gr.Dropdown(
                    choices=["Cherbourg", "Queenstown", "Southampton", "Bilinmiyor"], 
                    label="🛳️ Bindiği Liman", 
                    value="Southampton",
                    info="Gemiye bindiğiniz liman"
                )
                
                predict_button = gr.Button("🔮 Tahmin Et", variant="secondary", size="lg")
        
            with gr.Column(scale=2):
                gr.Markdown("## 📊 Tahmin Sonuçları")
                
                with gr.Row():
                    survival_percentage = gr.Textbox(
                        label="Hayatta Kalma Şansı", 
                        interactive=False
                    )
                
                result_text = gr.Markdown("")
                result_plot = gr.Plot(label="Tahmin Grafiği")
        
        predict_button.click(
            fn=predict_survival,
            inputs=[pclass, age, fare, sex, sibsp, parch, embarked],
            outputs=[result_text, result_plot, survival_percentage]
        )

# Ana arayüz
def create_interface():
    with gr.Blocks(title="Titanic Hayatta Kalma Tahmini", theme=gr.themes.Soft()) as interface:
        
        gr.Markdown("""
        # 🚢 Titanic Hayatta Kalma Tahmini
        ### Makine Öğrenmesi ile 1912 Titanic Faciasında Hayatta Kalma Şansınızı Hesaplayın
        """)
        
        with gr.Tabs():
            with gr.Tab("🔧 Model Eğitimi"):
                create_training_page()
                
            with gr.Tab("🔮 Tahmin Yap"):
                create_prediction_page()
        
        gr.Markdown("""
        ---
        ### 📝 Kullanım Kılavuzu:
        1. **Model Eğitimi** sekmesinden modeli eğitin
        2. **Tahmin Yap** sekmesine geçin
        3. Kişisel bilgilerinizi girin ve tahmin alın
        
        ### ℹ️ Not
        Bu uygulama 1912 Titanic verilerine dayalı bir tahmin modelidir. Sadece eğitim amaçlıdır.
        """)
    
    return interface

# Arayüzü başlat
if __name__ == "__main__":
    app = create_interface()
    app.launch(
        share=True,
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True,
        debug=True
    )

                                
                sibsp = gr.Slider(
                    minimum=0, 
                    maximum=8, 
                    value=0, 
                    label="👫 Kardeş/Eş Sayısı",
                    info="Gemideki kardeş veya eş sayısı"
                )
                
                parch = gr.Slider(
                    minimum=0, 
                    maximum=6, 
                    value=0, 
                    label="👶 Ebeveyn/Çocuk Sayısı",
                    info="Gemideki ebeveyn veya çocuk sayısı"
                )
                
                embarked = gr.Dropdown(
                    choices=["Cherbourg", "Queenstown", "Southampton", "Bilinmiyor"], 
                    label="🛳️ Bindiği Liman", 
                    value="Southampton",
                    info="Gemiye bindiğiniz liman"
                )
                
                predict_button = gr.Button("🔮 Tahmin Et", variant="secondary", size="lg")
        
        predict_button.click(
            fn=predict_survival,
            inputs=[pclass, age, fare, sex, sibsp, parch, embarked],
            outputs=[result_text, result_plot, survival_percentage]
        )

# Ana arayüz
def create_interface():
    with gr.Blocks(title="Titanic Hayatta Kalma Tahmini", theme=gr.themes.Soft()) as interface:
        
        gr.Markdown("""
        # 🚢 Titanic Hayatta Kalma Tahmini
        ### Makine Öğrenmesi ile 1912 Titanic Faciasında Hayatta Kalma Şansınızı Hesaplayın
        """)
        
        with gr.Tabs():
            with gr.Tab("🔧 Model Eğitimi"):
                create_training_page()
                
            with gr.Tab("🔮 Tahmin Yap"):
                create_prediction_page()
        
        gr.Markdown("""
        ---
        ### 📝 Kullanım Kılavuzu:
        1. **Model Eğitimi** sekmesinden modeli eğitin
        2. **Tahmin Yap** sekmesine geçin
        3. Kişisel bilgilerinizi girin ve tahmin alın
        
        ### ℹ️ Not
        Bu uygulama 1912 Titanic verilerine dayalı bir tahmin modelidir. Sadece eğitim amaçlıdır.
        """)
    
    return interface

# Arayüzü başlat
if __name__ == "__main__":
    app = create_interface()
    app.launch(
        share=True,
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True,
        debug=True
    )

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 415)